### Przykładowe problemy związane ze skalowalnością zadań ML:

Ograniczenie CPU: Dane mieszczą się w pamięci RAM, ale proces uczenia trwa za długo. Np. W przypadku konieczności sprawdzenia wielu kombinacji parametrów modelu, wielu modeli, itd. 


Ograniczenia pamięci: Dane są na tyle duże, że nie mieszczą się w pamięci RAM.


#### Pipeline

![](../img/ml-Pipeline.png)

#### Pipeline Model

![](../img/ml-PipelineModel.png)

### Potok przetwarzania ML

* <b>DataFrame</b>: interfejs API ML używa DataFrame, w którym można przechowywać różne typy danych. Na przykład DataFrame może mieć różne kolumny przechowujące tekst, wektory cech, prawdziwe etykiety i prognozy.

* <b>Transformer</b>: Transformator to algorytm, który może przekształcić jedną ramkę danych w inną ramkę danych. Na przykład model ML to transformator, który przekształca ramkę danych z funkcjami w ramkę danych z prognozami. Innym przykładem transformatora jest StringIndexer, który koduje zmienne tekstowe jako zmienne całkowito liczbowe.


* <b>Estimator</b>: Estymator to algorytm, który który można zaaplikować do DataFrame w celu wytworzenia transformatora. Np. Algorytm uczenia się jest estymatorem, który trenuje się na DataFrame i tworzy model.


* <b>Pipeline</b>: Potok przetwarzania łączy wiele transformatorów i estymatorów razem, aby określić przepływ pracy ML.


* <b>Parametr</b>: Wszystkie transformatory i estymatory mają teraz wspólny interfejs API do określania parametrów.

### Wczytanie danych

In [1]:
import os
user_name = 'jovyan'
print(user_name)

jovyan


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config("spark.executor.instances", "4")\
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

In [3]:
# ścieżka dostępu do pliku 
csv_path = 'file:////home/jovyan/data/2020/survey_results_public.csv'

In [4]:
db_name = user_name.replace('-','_')

In [5]:
spark.sql(f'DROP DATABASE IF EXISTS {db_name} CASCADE')
spark.sql(f'CREATE DATABASE {db_name}')
spark.sql(f'USE {db_name}')

DataFrame[]

In [6]:
table_name = "survey_2020"            

In [7]:
spark.sql(f'DROP TABLE IF EXISTS {table_name}')

spark.sql(f'CREATE TABLE IF NOT EXISTS {table_name} \
          USING csv \
          OPTIONS (HEADER true, INFERSCHEMA true, NULLVALUE "NA") \
          LOCATION "{csv_path}"')

DataFrame[]

In [8]:
spark.sql(f'describe {table_name}').show(100)

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Respondent|      int|   null|
|          MainBranch|   string|   null|
|            Hobbyist|   string|   null|
|                 Age|   double|   null|
|          Age1stCode|   string|   null|
|            CompFreq|   string|   null|
|           CompTotal|   double|   null|
|       ConvertedComp|   double|   null|
|             Country|   string|   null|
|        CurrencyDesc|   string|   null|
|      CurrencySymbol|   string|   null|
|DatabaseDesireNex...|   string|   null|
|  DatabaseWorkedWith|   string|   null|
|             DevType|   string|   null|
|             EdLevel|   string|   null|
|          Employment|   string|   null|
|           Ethnicity|   string|   null|
|              Gender|   string|   null|
|          JobFactors|   string|   null|
|              JobSat|   string|   null|
|             JobSeek|   string|   null|
|LanguageDesireN

### Przygotowanie danych do analizy

W ramach zadania chcemy stworzyć klasyfikator, który będzie przewidywać czy respondent zarabia więcej niż 60000 USD rocznie.

In [9]:
spark_df= spark.sql(f'SELECT *, CAST((convertedComp > 60000) AS STRING) AS compAboveAvg \
                    FROM {table_name} where convertedComp IS NOT NULL ')
spark_df.limit(5).toPandas()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro,compAboveAvg
0,8,I am a developer by profession,Yes,36.0,12,Yearly,116000.0,116000.0,United States,United States dollar,...,Appropriate in length,No,"Computer science, computer engineering, or sof...",Django;React.js;Vue.js,Flask,Just as welcome now as I felt last year,39.0,17,13,true
1,10,I am a developer by profession,Yes,22.0,14,Yearly,25000.0,32315.0,United Kingdom,Pound sterling,...,Appropriate in length,No,Mathematics or statistics,Flask;jQuery,Flask;jQuery,Somewhat more welcome now than last year,36.0,8,4,false
2,11,I am a developer by profession,Yes,23.0,13,Yearly,31000.0,40070.0,United Kingdom,Pound sterling,...,Appropriate in length,No,"Computer science, computer engineering, or sof...",Angular;Django;React.js,Angular;Angular.js;Django;React.js,Just as welcome now as I felt last year,40.0,10,2,false
3,12,I am a developer by profession,No,49.0,42,Monthly,1100.0,14268.0,Spain,European Euro,...,Appropriate in length,No,Mathematics or statistics,ASP.NET;jQuery,ASP.NET;jQuery,Just as welcome now as I felt last year,40.0,7,7,false
4,13,"I am not primarily a developer, but I write co...",Yes,53.0,14,Monthly,3000.0,38916.0,Netherlands,European Euro,...,Too long,No,None,None,None,A lot less welcome now than last year,36.0,35,20,false


<B>Dążymy do tego, żeby przygotować jeden wektor cech oraz jedną kolumnę z oznaczeniami.</B>

Kodujemy kolumny tekstowe na numeryczne oraz kodujemy wartości liczbowe na reprezentacje onehotencoder. Następnie dokonujemy asemblacji do jednego wektora.

In [10]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
# chcemy przewidziec compAboveAvg
y = 'compAboveAvg'
# na podstawie:
feature_columns = ['OpSys', 'EdLevel', 'MainBranch' , 'Country', 'JobSeek', 'YearsCode']

In [11]:
#Zaczynamy od transformatora StringIndexer, zamieniajacego wartosci 'string' na liczbe
# dla cech, ktore zostana wykorzystane do predykcji

##### najpierw pokazujemy prosta petle z FOR, a potem zrefactorujmy do list comprehension
stringindexer_stages_1 = []
for c in feature_columns:
    stringindexer_stages_1.append (StringIndexer(inputCol=c, outputCol='stringindexed_' + c).setHandleInvalid("keep"))

# i dla zmiennej objaśnianej
stringindexer_stages_1.append(StringIndexer(inputCol=y, outputCol='label'))


<b>handleInvalid</b> = How to handle invalid data during transform(). Options are 'keep' (invalid data presented as an extra categorical feature) or error (throw an error). 

In [12]:
# Refactoring do list comprehension
stringindexer_stages = [StringIndexer(inputCol=c, outputCol='stringindexed_' + c).setHandleInvalid("keep") for c in feature_columns]

# i dla zmiennej objaśnianej
stringindexer_stages += [StringIndexer(inputCol=y, outputCol='label')]
stringindexer_stages

[StringIndexer_022f972edae8,
 StringIndexer_66a3436d08a2,
 StringIndexer_3b84dfbe7ed7,
 StringIndexer_b77c894fe807,
 StringIndexer_b99314afef6f,
 StringIndexer_7bf2bc0aac3d,
 StringIndexer_cacc74c04860]

In [13]:
# Po wykonaniu takiej transformacji do DF zostaje dodane  7 nowych kolumn z prefixem "stringindexed_"
Pipeline(stages=stringindexer_stages).fit(spark_df).transform(spark_df).toPandas()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,YearsCode,YearsCodePro,compAboveAvg,stringindexed_OpSys,stringindexed_EdLevel,stringindexed_MainBranch,stringindexed_Country,stringindexed_JobSeek,stringindexed_YearsCode,label
0,8,I am a developer by profession,Yes,36.0,12,Yearly,116000.0,116000.0,United States,United States dollar,...,17,13,true,2.0,0.0,0.0,0.0,0.0,17.0,1.0
1,10,I am a developer by profession,Yes,22.0,14,Yearly,25000.0,32315.0,United Kingdom,Pound sterling,...,8,4,false,0.0,1.0,0.0,2.0,0.0,1.0,0.0
2,11,I am a developer by profession,Yes,23.0,13,Yearly,31000.0,40070.0,United Kingdom,Pound sterling,...,10,2,false,0.0,0.0,0.0,2.0,2.0,0.0,0.0
3,12,I am a developer by profession,No,49.0,42,Monthly,1100.0,14268.0,Spain,European Euro,...,7,7,false,0.0,2.0,0.0,10.0,0.0,3.0,0.0
4,13,"I am not primarily a developer, but I write co...",Yes,53.0,14,Monthly,3000.0,38916.0,Netherlands,European Euro,...,35,20,false,1.0,3.0,1.0,7.0,1.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34751,65619,"I am not primarily a developer, but I write co...",Yes,NaN,19,Monthly,30000.0,984.0,Nigeria,Nigerian naira,...,3,2,false,0.0,3.0,1.0,38.0,3.0,13.0,0.0
34752,65625,I am a developer by profession,Yes,NaN,17,Monthly,5500000.0,19428.0,Colombia,Colombian peso,...,12,5,false,4.0,0.0,0.0,39.0,1.0,7.0,0.0
34753,65629,I am a developer by profession,Yes,41.0,15,Yearly,200.0,200.0,United States,United States dollar,...,25,20,false,1.0,2.0,0.0,0.0,0.0,14.0,0.0
34754,65630,I am a developer by profession,Yes,NaN,17,Monthly,1000000.0,15048.0,Chile,Chilean peso,...,7,3,false,4.0,0.0,0.0,48.0,0.0,3.0,0.0


In [14]:
onehotencoder_stages = [OneHotEncoder(inputCol='stringindexed_' + c, outputCol='onehot_' + c) for c in feature_columns]

Rozbudowujemy pipeline:

Po wykonaniu takiej transformacji (stringIndexer+onehotencoder) do DF zostaje dodane  6 nowych kolumn z prefixem "onehot_".

In [15]:
pa = Pipeline(stages=stringindexer_stages + onehotencoder_stages).fit(spark_df).transform(spark_df).toPandas()

In [16]:
pa.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'Age', 'Age1stCode', 'CompFreq',
       'CompTotal', 'ConvertedComp', 'Country', 'CurrencyDesc',
       'CurrencySymbol', 'DatabaseDesireNextYear', 'DatabaseWorkedWith',
       'DevType', 'EdLevel', 'Employment', 'Ethnicity', 'Gender', 'JobFactors',
       'JobSat', 'JobSeek', 'LanguageDesireNextYear', 'LanguageWorkedWith',
       'MiscTechDesireNextYear', 'MiscTechWorkedWith',
       'NEWCollabToolsDesireNextYear', 'NEWCollabToolsWorkedWith', 'NEWDevOps',
       'NEWDevOpsImpt', 'NEWEdImpt', 'NEWJobHunt', 'NEWJobHuntResearch',
       'NEWLearn', 'NEWOffTopic', 'NEWOnboardGood', 'NEWOtherComms',
       'NEWOvertime', 'NEWPurchaseResearch', 'NEWPurpleLink', 'NEWSOSites',
       'NEWStuck', 'OpSys', 'OrgSize', 'PlatformDesireNextYear',
       'PlatformWorkedWith', 'PurchaseWhat', 'Sexuality', 'SOAccount',
       'SOComm', 'SOPartFreq', 'SOVisitFreq', 'SurveyEase', 'SurveyLength',
       'Trans', 'UndergradMajor', 'WebframeDesireNextYear',
  

Nowe kolumny zawieraja wartosci typu SparseVector zawierajacy mape bitowa.


In [17]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://miro.medium.com/max/2400/1*ggtP4a5YaRx6l09KQaYOnw.png")

In [18]:
print("Orginal values:")
print(pa['OpSys'].unique())
print ("---------")
print("StringIndexed values:")
print(pa['stringindexed_OpSys'].unique())
print ("---------")
print("OneHot values:")
print(pa['onehot_OpSys'].unique())

Orginal values:
['Linux-based' 'Windows' 'MacOS' None 'BSD']
---------
StringIndexed values:
[2. 0. 1. 4. 3.]
---------
OneHot values:
[SparseVector(4, {2: 1.0}) SparseVector(4, {0: 1.0})
 SparseVector(4, {1: 1.0}) SparseVector(4, {}) SparseVector(4, {3: 1.0})]


#### <B>Asemblacja</B> - połączenie wszystkich kolumn predykcyjnych do jednej (kolumna features)

In [19]:
extracted_columns = ['onehot_' + c for c in feature_columns]
vectorassembler_stage = VectorAssembler(inputCols=extracted_columns, outputCol='features') 

### Połączenie wszystkich krokþw przygotowania danych w jednym potoku przetwarzania (pipeline)

In [20]:
# wybór kolumn do ostatecznej ramki danych
# poza kolumnami features i label (które będą wykorzystywane do budowania modelu) 
# zostawiamy m.in. oryginalne kolumn (feature_columns) 
final_columns = [y] + feature_columns + extracted_columns + ['features', 'label']
final_columns

['compAboveAvg',
 'OpSys',
 'EdLevel',
 'MainBranch',
 'Country',
 'JobSeek',
 'YearsCode',
 'onehot_OpSys',
 'onehot_EdLevel',
 'onehot_MainBranch',
 'onehot_Country',
 'onehot_JobSeek',
 'onehot_YearsCode',
 'features',
 'label']

In [21]:
transformed_df = Pipeline(stages=stringindexer_stages + \
                          onehotencoder_stages + \
                          [vectorassembler_stage]).fit(spark_df).transform(spark_df).select(final_columns)

transformed_df.limit(5).toPandas()

,compAboveAvg,OpSys,EdLevel,MainBranch,Country,JobSeek,YearsCode,onehot_OpSys,onehot_EdLevel,onehot_MainBranch,onehot_Country,onehot_JobSeek,onehot_YearsCode,features,label
0,true,Linux-based,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",I am a developer by profession,United States,"I’m not actively looking, but I am open to new...",17,"(0.0, 0.0, 1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1.0
1,false,Windows,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",I am a developer by profession,United Kingdom,"I’m not actively looking, but I am open to new...",8,"(1.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0
2,false,Windows,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",I am a developer by profession,United Kingdom,I am actively looking for a job,10,"(1.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0
3,false,Windows,Some college/university study without earning ...,I am a developer by profession,Spain,"I’m not actively looking, but I am open to new...",7,"(1.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...",0.0
4,false,MacOS,"Secondary school (e.g. American high school, G...","I am not primarily a developer, but I write co...",Netherlands,I am not interested in new job opportunities,35,"(0.0, 1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 1.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0


### Podzial na zbior treningowy/testowy

In [22]:
training, test = transformed_df.randomSplit([0.8, 0.2], seed=1234)

In [23]:
training.count()

27890

### Uczenie modelu - model.fit()

In [24]:
# na poczatek wybierzemy drzewo decyzyjne. Nie musimy podawac zadnych parametrow
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol='features', labelCol='label')

In [25]:
simple_model = Pipeline(stages=[dt]).fit(training)

In [26]:
simple_model.stages[0]

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f4874f9c40a8, depth=5, numNodes=43, numClasses=2, numFeatures=229

### Predykcja - model.transform()

In [27]:
pred_simple = simple_model.transform(test)

In [28]:
show_columns = final_columns + ['prediction', 'rawPrediction', 'probability']
pred_simple.limit(5).select(show_columns).toPandas()

,compAboveAvg,OpSys,EdLevel,MainBranch,Country,JobSeek,YearsCode,onehot_OpSys,onehot_EdLevel,onehot_MainBranch,onehot_Country,onehot_JobSeek,onehot_YearsCode,features,label,prediction,rawPrediction,probability
0,false,None,"Associate degree (A.A., A.S., etc.)",I am a developer by profession,Germany,"I’m not actively looking, but I am open to new...",19,"(0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,1.0,"[725.0, 938.0]","[0.4359591100420926, 0.5640408899579074]"
1,false,None,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",I am a developer by profession,Germany,"I’m not actively looking, but I am open to new...",16,"(0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,1.0,"[725.0, 938.0]","[0.4359591100420926, 0.5640408899579074]"
2,false,None,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",I am a developer by profession,United States,"I’m not actively looking, but I am open to new...",15,"(0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,1.0,"[479.0, 5195.0]","[0.08442016214310892, 0.9155798378568911]"
3,false,None,"Professional degree (JD, MD, etc.)","I am not primarily a developer, but I write co...",Switzerland,"I’m not actively looking, but I am open to new...",8,"(0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,"[12474.0, 3277.0]","[0.7919497174782554, 0.20805028252174465]"
4,false,None,"Secondary school (e.g. American high school, G...",I am a developer by profession,United States,"I’m not actively looking, but I am open to new...",6,"(0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0,1.0,"[479.0, 5195.0]","[0.08442016214310892, 0.9155798378568911]"


## Ewaluacje

In [29]:
# macierz pomyłek (confusion matrix)
label_and_pred = pred_simple.select('label', 'prediction')
label_and_pred.groupBy('label', 'prediction').count().toPandas()

,label,prediction,count
0,1.0,1.0,2259
1,0.0,1.0,644
2,1.0,0.0,831
3,0.0,0.0,3132


In [30]:
# Ewaluator 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

In [31]:
auroc_simple = evaluator.evaluate(pred_simple)
auroc_simple

0.5986083542455158

In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_m = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator_m.evaluate(pred_simple)
accuracy

0.7851733177978445

## Dodanie hiperparametrów 

In [33]:
# Jakie wartości hiperparametru maxDepth mają być przetestowane:
from pyspark.ml.tuning import ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(dt.maxDepth, [2,3,4,5,6]).\
    build()

In [34]:
# Walidacja krzyżowa wykonwyana w celu optymalizacji hiperparametrów
from pyspark.ml.tuning import CrossValidator
cv = CrossValidator(estimator=dt, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=4)

In [35]:
# Budowa modelu na podstawie danych treningowych
cv_model = cv.fit(training)

In [36]:
cv_model.bestModel

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_f4874f9c40a8, depth=2, numNodes=5, numClasses=2, numFeatures=229

## Predykcja z nowym modelem

In [37]:
# Jak wygląda predykcja na zbiorze danych testowych?
pred_cv = cv_model.transform(test)
show_columns = final_columns + ['prediction', 'rawPrediction', 'probability']
pred_cv.limit(5).select(show_columns).toPandas()

,compAboveAvg,OpSys,EdLevel,MainBranch,Country,JobSeek,YearsCode,onehot_OpSys,onehot_EdLevel,onehot_MainBranch,onehot_Country,onehot_JobSeek,onehot_YearsCode,features,label,prediction,rawPrediction,probability
0,false,None,"Associate degree (A.A., A.S., etc.)",I am a developer by profession,Germany,"I’m not actively looking, but I am open to new...",19,"(0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",0.0,0.0,"[13897.0, 5414.0]","[0.7196416550152762, 0.28035834498472373]"
1,false,None,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",I am a developer by profession,Germany,"I’m not actively looking, but I am open to new...",16,"(0.0, 0.0, 0.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,"[13897.0, 5414.0]","[0.7196416550152762, 0.28035834498472373]"
2,false,None,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",I am a developer by profession,United States,"I’m not actively looking, but I am open to new...",15,"(0.0, 0.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0.0,1.0,"[656.0, 5837.0]","[0.10103188048667797, 0.898968119513322]"
3,false,None,"Professional degree (JD, MD, etc.)","I am not primarily a developer, but I write co...",Switzerland,"I’m not actively looking, but I am open to new...",8,"(0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0)","(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.0,0.0,"[13897.0, 5414.0]","[0.7196416550152762, 0.28035834498472373]"
4,false,None,"Secondary school (e.g. American high school, G...",I am a developer by profession,United States,"I’m not actively looking, but I am open to new...",6,"(0.0, 0.0, 0.0, 0.0)","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0)","(1.0, 0.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",0.0,1.0,"[656.0, 5837.0]","[0.10103188048667797, 0.898968119513322]"


In [38]:
# Confusion matrix
label_and_pred = pred_cv.select('label', 'prediction')
label_and_pred.groupBy('label', 'prediction').count().toPandas()

,label,prediction,count
0,1.0,1.0,1816
1,0.0,1.0,345
2,1.0,0.0,1274
3,0.0,0.0,3431


In [39]:
auroc_cv = evaluator.evaluate(pred_cv)
auroc_cv

0.6893249307498216

In [40]:
acc_cv = evaluator_m.evaluate(pred_cv)
acc_cv

0.7642004078065832

## Klasyfikacja za pomoca Gradient Boosted Trees

In [41]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(labelCol="label", featuresCol="features", maxIter=10)
model = gbt.fit(training)

In [42]:
evaluator.evaluate(model.transform(test))

0.8828084718336902

In [264]:
evaluator_m.evaluate(model.transform(test))

0.8049810661229245

## <span style='background:yellow'> ZADANIA - etap 2A</span>


Odpowiedz korzystając z <b>API DF</b> lub <b>SparkSQL</b>:
    
a) Ile jest unikalnych odpowiedzi w zapytaniu o poziom wykształcenia (EdLevel)?

b) Podaj średnią liczbę godzin przepracowywanych przez z respondentów pogrupowanych ze względu na kraj.

c) Narysuj wykres słupkowy popularności wykorzystywanych baz danych przez profesjonalnych programistów. Skorzystaj z funkcji split i posexplode. 

d)  Sprawdź czy można bardziej poprawić jakość predykcji dla zadania z tego notatnika: 
* dodając cechy?
* zmieniąjac model?
* lepiej dobierajac parametry nowego modelu? 

e) Notatnik zawierający wyniki umieść w repozytorium zespołu w branchu phase-2a.
